## Modeling - Classifying Book Cover by Genre

#### This notebook was run in Google Colab

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0

np.random.seed(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/datasets/capstone/images.zip

In [5]:
train = pd.read_csv('/content/drive/MyDrive/datasets/capstone/data/book30-listing-train.csv', encoding_errors='ignore', header=None)
train.columns = ['amazon_index', 'filename', 'image_url', 'title', 'author', 'category_id', 'category']

In [ ]:
train.head(2)

,amazon_index,filename,image_url,title,author,category_id,category
0,1588345297,1588345297.jpg,http://ecx.images-amazon.com/images/I/51l6XIoa...,With Schwarzkopf: Life Lessons of The Bear,Gus Lee,1,Biographies & Memoirs
1,1404803335,1404803335.jpg,http://ecx.images-amazon.com/images/I/51UJnL3T...,"Magnets: Pulling Together, Pushing Apart (Amaz...",Natalie M. Rosinsky,4,Children's Books


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/datasets/capstone/data/book30-listing-test.csv', encoding_errors='ignore', header=None)
test.columns = ['amazon_index', 'filename', 'image_url', 'title', 'author', 'category_id', 'category']

In [ ]:
test.head(2)

,amazon_index,filename,image_url,title,author,category_id,category
0,044310073X,044310073X.jpg,http://ecx.images-amazon.com/images/I/41kSLEos...,Oral and Maxillofacial Surgery: An Objective-B...,NaN,16,Medical Books
1,1438005687,1438005687.jpg,http://ecx.images-amazon.com/images/I/510l0qhi...,"Barron's GRE, 21st Edition",Sharon Weiner Green M.A.,28,Test Preparation


In [13]:
train_cleaned = pd.read_csv('/content/drive/MyDrive/datasets/capstone/data/train_cleaned.csv', index_col=0)
train_cleaned.head(3)

,amazon_index,filename,image_url,title,author,category_id,category
0,1588345297,1588345297.jpg,http://ecx.images-amazon.com/images/I/51l6XIoa...,With Schwarzkopf: Life Lessons of The Bear,Gus Lee,1,Biographies & Memoirs
1,1404803335,1404803335.jpg,http://ecx.images-amazon.com/images/I/51UJnL3T...,"Magnets: Pulling Together, Pushing Apart (Amaz...",Natalie M. Rosinsky,4,Children's Books
15,471310115,0471310115.jpg,http://ecx.images-amazon.com/images/I/61JkH6Io...,"Janice VanCleave's 201 Awesome, Magical, Bizar...",Janice VanCleave,4,Children's Books


In [12]:
test_cleaned = pd.read_csv('/content/drive/MyDrive/datasets/capstone/data/test_cleaned.csv', index_col=0)
test_cleaned.head(3)

,amazon_index,filename,image_url,title,author,category_id,category
2,60750715,0060750715.jpg,http://ecx.images-amazon.com/images/I/51p5aUY%...,George Balanchine: The Ballet Maker (Eminent L...,Robert Gottlieb,1,Biographies & Memoirs
5,312556411,0312556411.jpg,http://ecx.images-amazon.com/images/I/51uNnIlI...,Literature and Its Writers: A Compact Introduc...,Ann Charters,15,Literature & Fiction
9,545700272,0545700272.jpg,http://ecx.images-amazon.com/images/I/51KyLXcz...,A Handful of Stars,Cynthia Lord,4,Children's Books


#### Baseline Model

In [14]:
train['category'].value_counts()

Biographies & Memoirs           1710
Children's Books                1710
Science Fiction & Fantasy       1710
Calendars                       1710
Self-Help                       1710
Teen & Young Adult              1710
Reference                       1710
Comics & Graphic Novels         1710
History                         1710
Romance                         1710
Parenting & Relationships       1710
Arts & Photography              1710
Test Preparation                1710
Computers & Technology          1710
Law                             1710
Humor & Entertainment           1710
Literature & Fiction            1710
Religion & Spirituality         1710
Crafts, Hobbies & Home          1710
Politics & Social Sciences      1710
Cookbooks, Food & Wine          1710
Business & Money                1710
Travel                          1710
Science & Math                  1710
Medical Books                   1710
Health, Fitness & Dieting       1710
Sports & Outdoors               1710
C

In [15]:
train_cleaned['category'].value_counts()

Biographies & Memoirs           1710
Children's Books                1710
Literature & Fiction            1710
Romance                         1710
Teen & Young Adult              1710
Science Fiction & Fantasy       1710
Mystery, Thriller & Suspense    1710
Name: category, dtype: int64

#### Importing Images

In [16]:
datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_gen = datagen.flow_from_dataframe(
    dataframe=train,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256,256)
)

test_gen = datagen.flow_from_dataframe(
    dataframe=test,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256,256)
)

Found 51300 validated image filenames belonging to 30 classes.
Found 5700 validated image filenames belonging to 30 classes.


In [17]:
train_gen_2 = datagen.flow_from_dataframe(
    dataframe=train_cleaned,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256,256)
)

test_gen_2 = datagen.flow_from_dataframe(
    dataframe=test_cleaned,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(256,256)
)

Found 11970 validated image filenames belonging to 7 classes.
Found 1330 validated image filenames belonging to 7 classes.


#### Creating First Model with All 30 Categories

In [ ]:
model1 = Sequential()
model1.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Conv2D(64,(3,3),activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())
model1.add(Dense(64,activation='relu'))
model1.add(Dense(30,activation='softmax'))

model1.compile(
    optimizer ='adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history1 = model1.fit(
    train_gen,
    validation_data=test_gen,
    epochs=5
)

Epoch 1/5
1604/1604 [==============================] - 2367s 1s/step - loss: 3.2946 - accuracy: 0.0812 - val_loss: 3.1955 - val_accuracy: 0.1061
Epoch 2/5
1604/1604 [==============================] - 2226s 1s/step - loss: 2.9311 - accuracy: 0.1827 - val_loss: 3.1683 - val_accuracy: 0.1326
Epoch 3/5
1604/1604 [==============================] - 2292s 1s/step - loss: 2.0507 - accuracy: 0.4130 - val_loss: 3.7401 - val_accuracy: 0.1263
Epoch 4/5
1604/1604 [==============================] - 2207s 1s/step - loss: 0.9106 - accuracy: 0.7347 - val_loss: 5.9865 - val_accuracy: 0.1135
Epoch 5/5
1604/1604 [==============================] - 2248s 1s/step - loss: 0.3363 - accuracy: 0.9068 - val_loss: 8.8365 - val_accuracy: 0.1088


#### Creating First Model with 7 Categories

In [20]:
model1_1 = Sequential()
model1_1.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model1_1.add(MaxPooling2D(pool_size=(2,2)))

model1_1.add(Conv2D(64,(3,3),activation='relu'))
model1_1.add(MaxPooling2D(pool_size=(2,2)))

model1_1.add(Flatten())
model1_1.add(Dense(64,activation='relu'))
model1_1.add(Dense(7,activation='softmax'))

model1_1.compile(
    optimizer ='adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [21]:
history1_1 = model1_1.fit(
    train_gen_2,
    validation_data=test_gen_2,
    epochs=5
)

Epoch 1/5
375/375 [==============================] - 39s 99ms/step - loss: 2.0623 - accuracy: 0.2042 - val_loss: 1.8926 - val_accuracy: 0.2256
Epoch 2/5
375/375 [==============================] - 37s 98ms/step - loss: 1.7273 - accuracy: 0.3299 - val_loss: 1.9176 - val_accuracy: 0.2331
Epoch 3/5
375/375 [==============================] - 38s 102ms/step - loss: 1.1719 - accuracy: 0.5742 - val_loss: 2.3995 - val_accuracy: 0.2617
Epoch 4/5
375/375 [==============================] - 37s 99ms/step - loss: 0.5308 - accuracy: 0.8201 - val_loss: 3.9224 - val_accuracy: 0.2444
Epoch 5/5
375/375 [==============================] - 37s 99ms/step - loss: 0.2430 - accuracy: 0.9299 - val_loss: 5.5387 - val_accuracy: 0.2504


#### Repeating with Dropout Layer and Early Stopping for 30 Categories

In [ ]:
model2 = Sequential()
model2.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Conv2D(64,(3,3),activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Flatten())
model2.add(Dense(64,activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(30,activation='softmax'))

model2.compile(
    optimizer =Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
)

In [ ]:
history2 = model2.fit(
    train_gen,
    validation_data=test_gen,
    epochs=5,
    callbacks=[early_stop]
)

Epoch 1/5
1604/1604 [==============================] - 172s 99ms/step - loss: 3.4032 - accuracy: 0.0325 - val_loss: 3.4012 - val_accuracy: 0.0333
Epoch 2/5
1604/1604 [==============================] - 158s 98ms/step - loss: 3.4012 - accuracy: 0.0313 - val_loss: 3.4012 - val_accuracy: 0.0333
Epoch 3/5
1604/1604 [==============================] - 159s 99ms/step - loss: 3.4007 - accuracy: 0.0317 - val_loss: 3.4012 - val_accuracy: 0.0332
Epoch 4/5
1604/1604 [==============================] - 158s 98ms/step - loss: 3.4003 - accuracy: 0.0327 - val_loss: 3.4012 - val_accuracy: 0.0333
Epoch 5/5
1604/1604 [==============================] - 166s 103ms/step - loss: 3.4002 - accuracy: 0.0312 - val_loss: 3.4011 - val_accuracy: 0.0333


#### Repeating with Dropout Layer and Early Stopping for 7 Categories

In [22]:
model2_2 = Sequential()
model2_2.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model2_2.add(MaxPooling2D(pool_size=(2,2)))

model2_2.add(Conv2D(64,(3,3),activation='relu'))
model2_2.add(MaxPooling2D(pool_size=(2,2)))

model2_2.add(Flatten())
model2_2.add(Dense(64,activation='relu'))
model2_2.add(Dropout(0.2))
model2_2.add(Dense(7,activation='softmax'))

model2_2.compile(
    optimizer =Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [23]:
early_stop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
)

In [24]:
history2_2 = model2_2.fit(
    train_gen_2,
    validation_data=test_gen_2,
    epochs=5,
    callbacks=[early_stop]
)

Epoch 1/5
375/375 [==============================] - 38s 100ms/step - loss: 1.9352 - accuracy: 0.1753 - val_loss: 1.9100 - val_accuracy: 0.1910
Epoch 2/5
375/375 [==============================] - 38s 100ms/step - loss: 1.8843 - accuracy: 0.2188 - val_loss: 1.8661 - val_accuracy: 0.2248
Epoch 3/5
375/375 [==============================] - 44s 118ms/step - loss: 1.8131 - accuracy: 0.2660 - val_loss: 1.8409 - val_accuracy: 0.2714
Epoch 4/5
375/375 [==============================] - 38s 100ms/step - loss: 1.7174 - accuracy: 0.3193 - val_loss: 1.8230 - val_accuracy: 0.2880
Epoch 5/5
375/375 [==============================] - 37s 99ms/step - loss: 1.6097 - accuracy: 0.3792 - val_loss: 1.8321 - val_accuracy: 0.2932


#### Adding Additional Conv2D and MaxPooling2D Layer for 30 Categories

In [ ]:
model3 = Sequential()
model3.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add(Conv2D(64,(3,3),activation='relu'))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add(Conv2D(64,(3,3),activation='relu'))
model3.add(MaxPooling2D(pool_size=(2,2)))

model3.add(Flatten())
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(30,activation='softmax'))

model3.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history3 = model3.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 168s 104ms/step - loss: 3.3728 - accuracy: 0.0549 - val_loss: 3.3215 - val_accuracy: 0.0737
Epoch 2/10
1604/1604 [==============================] - 166s 103ms/step - loss: 3.2717 - accuracy: 0.0894 - val_loss: 3.2081 - val_accuracy: 0.1089
Epoch 3/10
1604/1604 [==============================] - 166s 103ms/step - loss: 3.1405 - accuracy: 0.1238 - val_loss: 3.1089 - val_accuracy: 0.1377
Epoch 4/10
1604/1604 [==============================] - 168s 104ms/step - loss: 3.0136 - accuracy: 0.1523 - val_loss: 3.0835 - val_accuracy: 0.1474
Epoch 5/10
1604/1604 [==============================] - 166s 103ms/step - loss: 2.8831 - accuracy: 0.1859 - val_loss: 3.0805 - val_accuracy: 0.1470
Epoch 6/10
1604/1604 [==============================] - 166s 104ms/step - loss: 2.7469 - accuracy: 0.2188 - val_loss: 3.0672 - val_accuracy: 0.1567
Epoch 7/10
1604/1604 [==============================] - 167s 104ms/step - loss: 2.5979 - accuracy: 0.2523 - val_

#### Adding Additional Conv2D and MaxPooling2D Layer for 7 Categories

In [25]:
model3_3 = Sequential()
model3_3.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model3_3.add(MaxPooling2D(pool_size=(2,2)))

model3_3.add(Conv2D(64,(3,3),activation='relu'))
model3_3.add(MaxPooling2D(pool_size=(2,2)))

model3_3.add(Conv2D(64,(3,3),activation='relu'))
model3_3.add(MaxPooling2D(pool_size=(2,2)))

model3_3.add(Flatten())
model3_3.add(Dense(64,activation='relu'))
model3_3.add(Dropout(0.2))
model3_3.add(Dense(7,activation='softmax'))

model3_3.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [26]:
history3_3 = model3_3.fit(
    train_gen_2,
    validation_data=test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 40s 106ms/step - loss: 1.9198 - accuracy: 0.1882 - val_loss: 1.8878 - val_accuracy: 0.2128
Epoch 2/10
375/375 [==============================] - 47s 127ms/step - loss: 1.8484 - accuracy: 0.2512 - val_loss: 1.8563 - val_accuracy: 0.2609
Epoch 3/10
375/375 [==============================] - 39s 103ms/step - loss: 1.7948 - accuracy: 0.2915 - val_loss: 1.7917 - val_accuracy: 0.3195
Epoch 4/10
375/375 [==============================] - 39s 103ms/step - loss: 1.7243 - accuracy: 0.3292 - val_loss: 1.7849 - val_accuracy: 0.3098
Epoch 5/10
375/375 [==============================] - 39s 104ms/step - loss: 1.6569 - accuracy: 0.3632 - val_loss: 1.7846 - val_accuracy: 0.3150
Epoch 6/10
375/375 [==============================] - 39s 103ms/step - loss: 1.5657 - accuracy: 0.4019 - val_loss: 1.7786 - val_accuracy: 0.3346
Epoch 7/10
375/375 [==============================] - 39s 104ms/step - loss: 1.4567 - accuracy: 0.4557 - val_loss: 1.7843 - val_ac

#### Adding Additional Conv2D and MaxPooling2D Layer for 30 Categories

In [ ]:
model4 = Sequential()
model4.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model4.add(MaxPooling2D(pool_size=(2,2)))

model4.add(Conv2D(64,(3,3),activation='relu'))
model4.add(MaxPooling2D(pool_size=(2,2)))

model4.add(Conv2D(64,(3,3),activation='relu'))
model4.add(MaxPooling2D(pool_size=(2,2)))

model4.add(Conv2D(64,(3,3),activation='relu'))
model4.add(MaxPooling2D(pool_size=(2,2)))

model4.add(Flatten())
model4.add(Dense(64,activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(30,activation='softmax'))

model4.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history4 = model4.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 169s 105ms/step - loss: 3.3503 - accuracy: 0.0603 - val_loss: 3.3006 - val_accuracy: 0.0737
Epoch 2/10
1604/1604 [==============================] - 167s 104ms/step - loss: 3.2256 - accuracy: 0.1001 - val_loss: 3.1514 - val_accuracy: 0.1325
Epoch 3/10
1604/1604 [==============================] - 168s 105ms/step - loss: 3.1134 - accuracy: 0.1323 - val_loss: 3.0589 - val_accuracy: 0.1546
Epoch 4/10
1604/1604 [==============================] - 167s 104ms/step - loss: 3.0150 - accuracy: 0.1598 - val_loss: 3.0205 - val_accuracy: 0.1679
Epoch 5/10
1604/1604 [==============================] - 168s 104ms/step - loss: 2.9293 - accuracy: 0.1809 - val_loss: 3.0128 - val_accuracy: 0.1625
Epoch 6/10
1604/1604 [==============================] - 168s 105ms/step - loss: 2.8463 - accuracy: 0.2023 - val_loss: 2.9912 - val_accuracy: 0.1744
Epoch 7/10
1604/1604 [==============================] - 169s 105ms/step - loss: 2.7561 - accuracy: 0.2229 - val_

#### Adding Additional Conv2D and MaxPooling2D Layer for 7 Categories

In [27]:
model4_4 = Sequential()
model4_4.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model4_4.add(MaxPooling2D(pool_size=(2,2)))

model4_4.add(Conv2D(64,(3,3),activation='relu'))
model4_4.add(MaxPooling2D(pool_size=(2,2)))

model4_4.add(Conv2D(64,(3,3),activation='relu'))
model4_4.add(MaxPooling2D(pool_size=(2,2)))

model4_4.add(Conv2D(64,(3,3),activation='relu'))
model4_4.add(MaxPooling2D(pool_size=(2,2)))

model4_4.add(Flatten())
model4_4.add(Dense(64,activation='relu'))
model4_4.add(Dropout(0.2))
model4_4.add(Dense(7,activation='softmax'))

model4_4.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [28]:
history4_4 = model4_4.fit(
    train_gen_2,
    validation_data=test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 43s 112ms/step - loss: 1.9066 - accuracy: 0.2016 - val_loss: 1.8646 - val_accuracy: 0.2511
Epoch 2/10
375/375 [==============================] - 39s 104ms/step - loss: 1.8378 - accuracy: 0.2630 - val_loss: 1.8243 - val_accuracy: 0.2850
Epoch 3/10
375/375 [==============================] - 40s 106ms/step - loss: 1.7842 - accuracy: 0.3049 - val_loss: 1.7697 - val_accuracy: 0.3098
Epoch 4/10
375/375 [==============================] - 39s 104ms/step - loss: 1.7330 - accuracy: 0.3327 - val_loss: 1.7850 - val_accuracy: 0.3098
Epoch 5/10
375/375 [==============================] - 39s 104ms/step - loss: 1.6797 - accuracy: 0.3607 - val_loss: 1.7455 - val_accuracy: 0.3368
Epoch 6/10
375/375 [==============================] - 39s 104ms/step - loss: 1.6148 - accuracy: 0.3880 - val_loss: 1.7207 - val_accuracy: 0.3594
Epoch 7/10
375/375 [==============================] - 39s 104ms/step - loss: 1.5585 - accuracy: 0.4104 - val_loss: 1.7123 - val_ac

#### Adding Additional Dense Layers After Flatten for 30 Categories

In [ ]:
model5 = Sequential()
model5.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model5.add(MaxPooling2D(pool_size=(2,2)))

model5.add(Conv2D(64,(3,3),activation='relu'))
model5.add(MaxPooling2D(pool_size=(2,2)))

model5.add(Conv2D(64,(3,3),activation='relu'))
model5.add(MaxPooling2D(pool_size=(2,2)))

model5.add(Conv2D(64,(3,3),activation='relu'))
model5.add(MaxPooling2D(pool_size=(2,2)))

model5.add(Flatten())
model5.add(Dense(64,activation='relu'))
model5.add(Dense(64,activation='relu'))
model5.add(Dense(64,activation='relu'))
model5.add(Dropout(0.2))
model5.add(Dense(30,activation='softmax'))

model5.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history5 = model5.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 169s 105ms/step - loss: 3.3711 - accuracy: 0.0502 - val_loss: 3.3088 - val_accuracy: 0.0702
Epoch 2/10
1604/1604 [==============================] - 168s 105ms/step - loss: 3.2625 - accuracy: 0.0839 - val_loss: 3.2223 - val_accuracy: 0.0967
Epoch 3/10
1604/1604 [==============================] - 168s 105ms/step - loss: 3.1581 - accuracy: 0.1122 - val_loss: 3.1011 - val_accuracy: 0.1311
Epoch 4/10
1604/1604 [==============================] - 169s 105ms/step - loss: 3.0753 - accuracy: 0.1363 - val_loss: 3.0670 - val_accuracy: 0.1486
Epoch 5/10
1604/1604 [==============================] - 168s 105ms/step - loss: 3.0071 - accuracy: 0.1547 - val_loss: 2.9991 - val_accuracy: 0.1563
Epoch 6/10
1604/1604 [==============================] - 169s 105ms/step - loss: 2.9378 - accuracy: 0.1756 - val_loss: 2.9962 - val_accuracy: 0.1591
Epoch 7/10
1604/1604 [==============================] - 168s 105ms/step - loss: 2.8746 - accuracy: 0.1917 - val_

#### Adding Additional Dense Layers After Flatten for 7 Categories

In [29]:
model5_5 = Sequential()
model5_5.add(Conv2D(64,(3,3),activation='relu',input_shape=(256,256,3)))
model5_5.add(MaxPooling2D(pool_size=(2,2)))

model5_5.add(Conv2D(64,(3,3),activation='relu'))
model5_5.add(MaxPooling2D(pool_size=(2,2)))

model5_5.add(Conv2D(64,(3,3),activation='relu'))
model5_5.add(MaxPooling2D(pool_size=(2,2)))

model5_5.add(Conv2D(64,(3,3),activation='relu'))
model5_5.add(MaxPooling2D(pool_size=(2,2)))

model5_5.add(Flatten())
model5_5.add(Dense(64,activation='relu'))
model5_5.add(Dense(64,activation='relu'))
model5_5.add(Dense(64,activation='relu'))
model5_5.add(Dropout(0.2))
model5_5.add(Dense(7,activation='softmax'))

model5_5.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [30]:
history5_5 = model5_5.fit(
    train_gen_2,
    validation_data=test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 43s 112ms/step - loss: 1.9183 - accuracy: 0.1893 - val_loss: 1.8676 - val_accuracy: 0.2444
Epoch 2/10
375/375 [==============================] - 47s 125ms/step - loss: 1.8616 - accuracy: 0.2456 - val_loss: 1.8452 - val_accuracy: 0.2774
Epoch 3/10
375/375 [==============================] - 40s 107ms/step - loss: 1.8169 - accuracy: 0.2797 - val_loss: 1.8108 - val_accuracy: 0.2782
Epoch 4/10
375/375 [==============================] - 39s 105ms/step - loss: 1.7815 - accuracy: 0.3003 - val_loss: 1.7804 - val_accuracy: 0.3135
Epoch 5/10
375/375 [==============================] - 39s 105ms/step - loss: 1.7316 - accuracy: 0.3297 - val_loss: 1.7602 - val_accuracy: 0.3271
Epoch 6/10
375/375 [==============================] - 39s 105ms/step - loss: 1.6762 - accuracy: 0.3612 - val_loss: 1.7526 - val_accuracy: 0.3286
Epoch 7/10
375/375 [==============================] - 39s 105ms/step - loss: 1.6144 - accuracy: 0.3891 - val_loss: 1.7877 - val_ac

#### Trying EfficientNet for 30 Categories

In [ ]:
eff_train_gen = datagen.flow_from_dataframe(
    dataframe=train,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(224,224)
)

eff_test_gen = datagen.flow_from_dataframe(
    dataframe=test,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(224,224)
)

Found 51300 validated image filenames belonging to 30 classes.
Found 5700 validated image filenames belonging to 30 classes.


In [ ]:
effnet = EfficientNetB0(include_top=False, weights='imagenet')

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:
effnet.trainable = False

In [ ]:
model6 = Sequential()
model6.add(effnet)
model6.add(GlobalAveragePooling2D())
model6.add(Dense(30, activation='softmax'))

model6.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history6 = model6.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 161s 97ms/step - loss: 3.1778 - accuracy: 0.1286 - val_loss: 3.0374 - val_accuracy: 0.1665
Epoch 2/10
1604/1604 [==============================] - 156s 97ms/step - loss: 2.9937 - accuracy: 0.1734 - val_loss: 2.9552 - val_accuracy: 0.1796
Epoch 3/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.9307 - accuracy: 0.1881 - val_loss: 2.9171 - val_accuracy: 0.1863
Epoch 4/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8954 - accuracy: 0.1968 - val_loss: 2.8931 - val_accuracy: 0.1923
Epoch 5/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8690 - accuracy: 0.2039 - val_loss: 2.8767 - val_accuracy: 0.1958
Epoch 6/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8527 - accuracy: 0.2076 - val_loss: 2.8653 - val_accuracy: 0.1984
Epoch 7/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8368 - accuracy: 0.2112 - val_loss: 2

#### Trying EfficientNet for 7 Categories

In [31]:
eff_train_gen_2 = datagen.flow_from_dataframe(
    dataframe=train_cleaned,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(224,224)
)

eff_test_gen_2 = datagen.flow_from_dataframe(
    dataframe=test_cleaned,
    directory='/content/224x224',
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=(224,224)
)

Found 11970 validated image filenames belonging to 7 classes.
Found 1330 validated image filenames belonging to 7 classes.


In [32]:
effnet = EfficientNetB0(include_top=False, weights='imagenet')

16719872/16705208 [==============================] - 0s 0us/step


In [33]:
effnet.trainable = False

In [34]:
model6_6 = Sequential()
model6_6.add(effnet)
model6_6.add(GlobalAveragePooling2D())
model6_6.add(Dense(7, activation='softmax'))

model6_6.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [36]:
history6_6 = model6_6.fit(
    eff_train_gen_2,
    validation_data=eff_test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 83s 110ms/step - loss: 1.9473 - accuracy: 0.1464 - val_loss: 1.9462 - val_accuracy: 0.1429
Epoch 2/10
375/375 [==============================] - 35s 94ms/step - loss: 1.9467 - accuracy: 0.1417 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 3/10
375/375 [==============================] - 30s 79ms/step - loss: 1.9469 - accuracy: 0.1419 - val_loss: 1.9460 - val_accuracy: 0.1286
Epoch 4/10
375/375 [==============================] - 30s 79ms/step - loss: 1.9462 - accuracy: 0.1481 - val_loss: 1.9464 - val_accuracy: 0.1429
Epoch 5/10
375/375 [==============================] - 29s 78ms/step - loss: 1.9472 - accuracy: 0.1477 - val_loss: 1.9466 - val_accuracy: 0.1429
Epoch 6/10
375/375 [==============================] - 30s 80ms/step - loss: 1.9466 - accuracy: 0.1425 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 7/10
375/375 [==============================] - 29s 78ms/step - loss: 1.9465 - accuracy: 0.1487 - val_loss: 1.9456 - val_accuracy

#### Adding Dense Layers and Dropout Layer to EfficientNet for 30 Categories

In [ ]:
model7 = Sequential()
model7.add(effnet)
model7.add(GlobalAveragePooling2D())
model7.add(Dense(64,activation='relu'))
model7.add(Dropout(0.2))
model7.add(Dense(30, activation='softmax'))

model7.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history7 = model7.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 163s 98ms/step - loss: 3.2287 - accuracy: 0.1054 - val_loss: 3.0501 - val_accuracy: 0.1596
Epoch 2/10
1604/1604 [==============================] - 154s 96ms/step - loss: 3.0390 - accuracy: 0.1570 - val_loss: 2.9516 - val_accuracy: 0.1816
Epoch 3/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.9700 - accuracy: 0.1752 - val_loss: 2.9073 - val_accuracy: 0.1932
Epoch 4/10
1604/1604 [==============================] - 153s 96ms/step - loss: 2.9331 - accuracy: 0.1840 - val_loss: 2.8820 - val_accuracy: 0.1928
Epoch 5/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.9044 - accuracy: 0.1920 - val_loss: 2.8641 - val_accuracy: 0.1956
Epoch 6/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8814 - accuracy: 0.1967 - val_loss: 2.8470 - val_accuracy: 0.2028
Epoch 7/10
1604/1604 [==============================] - 153s 95ms/step - loss: 2.8638 - accuracy: 0.2010 - val_loss: 2

In [ ]:
history7 = model7.fit(
    train_gen,
    validation_data=test_gen,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
1604/1604 [==============================] - 158s 99ms/step - loss: 2.8091 - accuracy: 0.2154 - val_loss: 2.8069 - val_accuracy: 0.2147
Epoch 2/20
1604/1604 [==============================] - 157s 98ms/step - loss: 2.7995 - accuracy: 0.2166 - val_loss: 2.8031 - val_accuracy: 0.2181
Epoch 3/20
1604/1604 [==============================] - 155s 97ms/step - loss: 2.7926 - accuracy: 0.2190 - val_loss: 2.7985 - val_accuracy: 0.2174
Epoch 4/20
1604/1604 [==============================] - 156s 97ms/step - loss: 2.7789 - accuracy: 0.2220 - val_loss: 2.7937 - val_accuracy: 0.2184
Epoch 5/20
1604/1604 [==============================] - 155s 97ms/step - loss: 2.7709 - accuracy: 0.2251 - val_loss: 2.7905 - val_accuracy: 0.2209
Epoch 6/20
1604/1604 [==============================] - 158s 99ms/step - loss: 2.7653 - accuracy: 0.2271 - val_loss: 2.7898 - val_accuracy: 0.2202
Epoch 7/20
1604/1604 [==============================] - 156s 97ms/step - loss: 2.7592 - accuracy: 0.2267 - val_loss: 2

In [ ]:
model8 = Sequential()
model8.add(effnet)
model8.add(GlobalAveragePooling2D())
model8.add(Dense(64,activation='relu'))
model8.add(Dense(64,activation='relu'))
model8.add(Dense(64,activation='relu'))
model8.add(Dropout(0.2))
model8.add(Dense(30, activation='softmax'))

model8.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history8 = model8.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
1604/1604 [==============================] - 163s 98ms/step - loss: 3.2530 - accuracy: 0.0935 - val_loss: 3.0641 - val_accuracy: 0.1546
Epoch 2/10
1604/1604 [==============================] - 155s 97ms/step - loss: 3.0494 - accuracy: 0.1486 - val_loss: 2.9470 - val_accuracy: 0.1784
Epoch 3/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.9747 - accuracy: 0.1694 - val_loss: 2.9078 - val_accuracy: 0.1870
Epoch 4/10
1604/1604 [==============================] - 155s 97ms/step - loss: 2.9291 - accuracy: 0.1801 - val_loss: 2.8776 - val_accuracy: 0.1911
Epoch 5/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8981 - accuracy: 0.1903 - val_loss: 2.8626 - val_accuracy: 0.1933
Epoch 6/10
1604/1604 [==============================] - 155s 97ms/step - loss: 2.8769 - accuracy: 0.1958 - val_loss: 2.8462 - val_accuracy: 0.2033
Epoch 7/10
1604/1604 [==============================] - 154s 96ms/step - loss: 2.8575 - accuracy: 0.2009 - val_loss: 2

#### Adding Dense Layers and Dropout Layer to EfficientNet for 7 Categories

In [37]:
model7_7 = Sequential()
model7_7.add(effnet)
model7_7.add(GlobalAveragePooling2D())
model7_7.add(Dense(64,activation='relu'))
model7_7.add(Dropout(0.2))
model7_7.add(Dense(7, activation='softmax'))

model7_7.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [38]:
history7_7 = model7_7.fit(
    eff_train_gen_2,
    validation_data=eff_test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 37s 84ms/step - loss: 1.9479 - accuracy: 0.1469 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 2/10
375/375 [==============================] - 31s 82ms/step - loss: 1.9470 - accuracy: 0.1434 - val_loss: 1.9457 - val_accuracy: 0.1429
Epoch 3/10
375/375 [==============================] - 31s 82ms/step - loss: 1.9461 - accuracy: 0.1442 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 4/10
375/375 [==============================] - 31s 83ms/step - loss: 1.9459 - accuracy: 0.1489 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 5/10
375/375 [==============================] - 31s 83ms/step - loss: 1.9463 - accuracy: 0.1427 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 6/10
375/375 [==============================] - 31s 82ms/step - loss: 1.9459 - accuracy: 0.1454 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 7/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9459 - accuracy: 0.1458 - val_loss: 1.9458 - val_accuracy:

In [39]:
history7_7 = model7_7.fit(
    eff_train_gen_2,
    validation_data=eff_test_gen_2,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
375/375 [==============================] - 30s 80ms/step - loss: 1.9459 - accuracy: 0.1399 - val_loss: 1.9457 - val_accuracy: 0.1436
Epoch 2/20
375/375 [==============================] - 30s 80ms/step - loss: 1.9454 - accuracy: 0.1491 - val_loss: 1.9456 - val_accuracy: 0.1684
Epoch 3/20
375/375 [==============================] - 30s 81ms/step - loss: 1.9456 - accuracy: 0.1477 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 4/20
375/375 [==============================] - 30s 80ms/step - loss: 1.9456 - accuracy: 0.1509 - val_loss: 1.9456 - val_accuracy: 0.1459
Epoch 5/20
375/375 [==============================] - 30s 81ms/step - loss: 1.9456 - accuracy: 0.1489 - val_loss: 1.9455 - val_accuracy: 0.1609
Epoch 6/20
375/375 [==============================] - 31s 82ms/step - loss: 1.9454 - accuracy: 0.1426 - val_loss: 1.9457 - val_accuracy: 0.1444
Epoch 7/20
375/375 [==============================] - 30s 81ms/step - loss: 1.9453 - accuracy: 0.1470 - val_loss: 1.9456 - val_accuracy:

In [40]:
model8_8 = Sequential()
model8_8.add(effnet)
model8_8.add(GlobalAveragePooling2D())
model8_8.add(Dense(64,activation='relu'))
model8_8.add(Dense(64,activation='relu'))
model8_8.add(Dense(64,activation='relu'))
model8_8.add(Dropout(0.2))
model8_8.add(Dense(7, activation='softmax'))

model8_8.compile(
    optimizer = Adam(learning_rate=.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [41]:
history8_8 = model8_8.fit(
    eff_train_gen_2,
    validation_data=eff_test_gen_2,
    epochs=10,
    callbacks=[early_stop]
)

Epoch 1/10
375/375 [==============================] - 38s 84ms/step - loss: 1.9466 - accuracy: 0.1399 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 2/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9464 - accuracy: 0.1400 - val_loss: 1.9462 - val_accuracy: 0.1429
Epoch 3/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9461 - accuracy: 0.1436 - val_loss: 1.9458 - val_accuracy: 0.1602
Epoch 4/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9463 - accuracy: 0.1419 - val_loss: 1.9458 - val_accuracy: 0.1436
Epoch 5/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9460 - accuracy: 0.1454 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 6/10
375/375 [==============================] - 30s 81ms/step - loss: 1.9459 - accuracy: 0.1474 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 7/10
375/375 [==============================] - 30s 80ms/step - loss: 1.9461 - accuracy: 0.1388 - val_loss: 1.9458 - val_accuracy: